In [1]:
import numpy as np

from modules.zapocet import * 

In [2]:
FI = 300  # mm
TW = 20  # mm
H = 50  # m
M = 1000  # kg
q = 1.5 # sucinitel duktility
ag = 0.63 # spickove zrychlenie
tvary = 3 # pocet tvarov
hmoty = 4 # pocet hmot
podlozie = "B"
typ_spektra = 2

In [3]:
f = vlastne_frekvencie([0.082, 0.496, 1.353])
print(f'f1 = {f[0]} hz \nf2 = {f[1]} hz \nf3 = {f[2]} hz')

f1 = 0.082 hz 
f2 = 0.496 hz 
f3 = 1.353 hz


In [4]:
omega = uhlove_frekvencie(f)
print(f'omega1 = {omega[0]:.3f} rad/s \nomega2 = {omega[1]:.3f} rad/s \nomega3 = {omega[2]:.3f} rad/s')

omega1 = 0.515 rad/s 
omega2 = 3.116 rad/s 
omega3 = 8.501 rad/s


In [5]:
T = vlastne_periody(f)
print(f'T1 = {T[0]:.3f} s \nT2 = {T[1]:.3f} s \nT3 = {T[2]:.3f} s')

T1 = 12.195 s 
T2 = 2.016 s 
T3 = 0.739 s


Normovane poradnice vlastnych tvarov

In [6]:
# 1. poradnice vlastny tvar
por_1 = [0.09433, 0.33232, 0.65079, 1.00000]

# 2. poradnice vlastny tvar
por_2 = [0.52683, 1.00000, 0.43530, -0.97353]

# 3. poradnice vlastny tvar
por_3 = [-0.94353, -0.28080, 0.89383, -0.58439]

In [7]:
poradnice = np.array([por_1, por_2, por_3])
poradnice

array([[ 0.09433,  0.33232,  0.65079,  1.     ],
       [ 0.52683,  1.     ,  0.4353 , -0.97353],
       [-0.94353, -0.2808 ,  0.89383, -0.58439]])

### Navrhove spektrum

In [8]:
parametre = spektrum_parametre(podlozie, typ_spektra)
parametre

{'S': 1.35, 'T_B': 0.05, 'T_C': 0.25, 'T_D': 1.2}

In [9]:
def spektrum(ag, q, periody):
    Sd = []
    for perioda in periody:
        Sd.append(sd(parametre, perioda, ag, q))
    print(f'Sd1 = {Sd[0]:.3f} \nSd2 = {Sd[1]:.3f} \nSd3 = {Sd[2]:.3f}')
    return Sd

In [10]:
Sd = spektrum(ag, q, T)

Sd1 = 0.126 
Sd2 = 0.126 
Sd3 = 0.479


In [11]:
def ni(poradnice, M, stlpec, riadok):
    i = stlpec
    j = riadok

    poradnice_sum = np.sum(poradnice, axis=1)
    citatel = poradnice_sum[j]*M*poradnice[j][i]
    temp = np.array(poradnice[j])**2*M
    menovatel = np.sum(temp)

    return citatel/menovatel

In [12]:
ni_ = []
for i in range(hmoty):
    stlpce = []
    for j in range(tvary):
        stlpce.append(ni(poradnice, M, stlpec=i, riadok=j))
    ni_.append(stlpce)

In [13]:
ni_ = np.array(ni_).T
ni_

array([[ 0.12701387,  0.44746368,  0.87627854,  1.34648434],
       [ 0.21568033,  0.40939266,  0.17820862, -0.39855603],
       [ 0.40920089,  0.12178056, -0.38764643,  0.25344495]])

In [14]:
def odozva_preTvar(ni, M, Sd, ag, tvar):
    return ni[tvar]*ag*M*Sd[tvar]

In [15]:
odozvy = []
for i in range(tvary):
    temp = odozva_preTvar(ni_, M, Sd, ag, tvar=i)
    print(f'{i+1}. vlastny tvar \nR{i+1} = {temp}')
    odozvy.append(temp)
odozvy = np.array(odozvy)

1. vlastny tvar 
R1 = [ 10.08236082  35.51966658  69.55899078 106.88392689]
2. vlastny tvar 
R2 = [ 17.12070494  32.49758924  14.1462006  -31.63737805]
3. vlastny tvar 
R3 = [ 123.60555698   36.78573061 -117.09469227   76.55702675]


In [16]:
for i in range(hmoty):
    V = np.sqrt(np.sum(odozvy[0][i:hmoty])**2+np.sum(odozvy[1][i:hmoty])**2+np.sum(odozvy[2][i:hmoty])**2)
    print(f'V{i+1} = {V:.3f} N')

V1 = 254.364 N
V2 = 212.526 N
V3 = 181.883 N
V4 = 135.226 N


In [17]:
ramena = H_vector(H)
print(f'r1 = {ramena[0]} m \nr2 = {ramena[1]} m \nr3 = {ramena[2]} m')

r1 = 12.5 m 
r2 = 25.0 m 
r3 = 37.5 m


In [18]:
def moment_preklopenia(M, ag, Sd, ni_, ramena, stlpec, riadok):
    i = stlpec
    j = riadok

    moment = M*ag*Sd[i]*ni_[i][j]*ramena[j]
    return moment

In [19]:
momenty_preklopenia = []
for j in range(hmoty):
    print(f'uzol {j+1}:')
    moment_v_uzle = []
    for i in range(tvary):
        temp = moment_preklopenia(M, ag, Sd, ni_, ramena, stlpec=i, riadok=j)
        print(f'Tvar{i+1} - M = {temp:.3f} Nm')
        moment_v_uzle.append(temp)
    momenty_preklopenia.append(moment_v_uzle)

uzol 1:
Tvar1 - M = 126.030 Nm
Tvar2 - M = 214.009 Nm
Tvar3 - M = 1545.069 Nm
uzol 2:
Tvar1 - M = 887.992 Nm
Tvar2 - M = 812.440 Nm
Tvar3 - M = 919.643 Nm
uzol 3:
Tvar1 - M = 2608.462 Nm
Tvar2 - M = 530.483 Nm
Tvar3 - M = -4391.051 Nm
uzol 4:
Tvar1 - M = 5344.196 Nm
Tvar2 - M = -1581.869 Nm
Tvar3 - M = 3827.851 Nm


In [20]:
def ohybove_momenty(odozvy, hmoty, ramena, tvary):
    for i in range(hmoty):
        odmocnit = []
        for j in range(tvary):
            odmocnit.append((np.dot(odozvy[j][i+1:hmoty], ramena[0:hmoty-i-1]))**2)
        
        print(f'M{i+2} = {np.sqrt(np.sum(odmocnit))/1000:.5f} kNm')

In [21]:
ohybove_momenty(odozvy, hmoty, ramena, tvary)

M2 = 6.21889 kNm
M3 = 3.62252 kNm
M4 = 1.69033 kNm
M5 = 0.00000 kNm


In [27]:
def ohybovy_moment_votknutie(momenty_preklopenia):
    M_sum = np.sum(momenty_preklopenia, axis=0)
    M_max = np.sqrt(M_sum[0]**2+M_sum[1]**2+M_sum[2]**2)/1000
    print(f'M_max = {M_max:.5f} kNm')

In [28]:
ohybovy_moment_votknutie(momenty_preklopenia)

M_max = 9.16612 kNm
